In [ ]:
import yfinance as yf
import pandas as pd
import time

# ✅ Full US Stock Universe (Sector-Wise 50 Stocks)
us_stock_universe = {
    "Technology": ["AAPL", "MSFT", "GOOGL", "NVDA", "META"],
    "Banking & Financials": ["JPM", "BAC", "WFC", "C", "GS"],
    "Consumer Goods": ["PG", "KO", "PEP", "MO", "MNST"],
    "Pharma & Healthcare": ["JNJ", "PFE", "MRNA", "LLY", "UNH"],
    "Energy": ["XOM", "CVX", "COP", "PSX", "SLB"],
    "Automobile": ["TSLA", "F", "GM", "RIVN", "LCID"],
    "Retail & E-commerce": ["AMZN", "WMT", "COST", "TGT", "HD"],
    "Aerospace & Defense": ["BA", "LMT", "NOC", "GD", "RTX"],
    "Telecom & Media": ["VZ", "T", "DIS", "NFLX", "CMCSA"],
    "Semiconductors": ["AMD", "TSM", "INTC", "AVGO", "QCOM"]
}

# ✅ Data Storage List
stock_data = []

# ✅ Fetch Data for Each Stock
for sector, stocks in us_stock_universe.items():
    for stock in stocks:
        try:
            ticker = yf.Ticker(stock)

            # ✅ Fetch Last Closing Price
            hist = ticker.history(period="1d")
            last_price = hist["Close"].iloc[-1] if not hist.empty else "N/A"

            # ✅ Fetch Current P/E Ratio
            current_pe = ticker.info.get("trailingPE", "N/A")

            # ✅ Fetch Historical Avg P/E Ratio (Past 3 Years)
            eps_values = ticker.financials.loc["Diluted EPS"].dropna() if "Diluted EPS" in ticker.financials.index else pd.Series()
            avg_pe = (last_price / eps_values).mean() if not eps_values.empty else "N/A"

            # ✅ Fetch Direct Net Worth (Shareholder's Equity)
            net_worth = ticker.balance_sheet.loc["Total Stockholder Equity"].iloc[0] if "Total Stockholder Equity" in ticker.balance_sheet.index else "N/A"

            # ✅ If Direct Net Worth is N/A, Approximate It
            if net_worth == "N/A":
                market_cap = ticker.info.get("marketCap", "N/A")
                price_to_book = ticker.info.get("priceToBook", "N/A")

                if market_cap != "N/A" and price_to_book != "N/A" and price_to_book > 0:
                    net_worth = market_cap / price_to_book  # **Estimate Net Worth**
                else:
                    net_worth = "N/A"

            # ✅ Convert Net Worth to Billion $
            net_worth_billion = round(net_worth / 1e9, 2) if net_worth != "N/A" else "N/A"

            # ✅ Store Data
            stock_data.append({
                "Sector": sector,
                "Stock": stock,
                "Last Price ($)": round(last_price, 2) if last_price != "N/A" else "N/A",
                "Current P/E Ratio": round(current_pe, 2) if current_pe != "N/A" else "N/A",
                "Historical Avg P/E Ratio": round(avg_pe, 2) if avg_pe != "N/A" else "N/A",
                "Net Worth ($B)": net_worth_billion
            })

            print(f"✅ {stock} - Price: {last_price}, P/E: {current_pe}, Avg P/E: {avg_pe}, Net Worth: {net_worth_billion}")

        except Exception as e:
            print(f"❌ Error fetching {stock}: {e}")

        time.sleep(2)  # ✅ API Rate Limit Avoid Karne ke liye

# ✅ Convert to Pandas DataFrame
df = pd.DataFrame(stock_data)

# ✅ Print as Table
print(df)

# ✅ Save to CSV File (Optional)
df.to_csv("stock_prices.csv", index=False)


In [ ]:
# Define stock universe
us_stock_universe = {
    "Technology": ["AAPL", "MSFT", "GOOGL", "NVDA", "META"],
    "Banking & Financials": ["JPM", "BAC", "WFC", "C", "GS"],
    "Consumer Goods": ["PG", "KO", "PEP", "MO", "MNST"],
    "Pharma & Healthcare": ["JNJ", "PFE", "MRNA", "LLY", "UNH"],
    "Energy": ["XOM", "CVX", "COP", "PSX", "SLB"],
    "Automobile": ["TSLA", "F", "GM", "RIVN", "LCID"],
    "Retail & E-commerce": ["AMZN", "WMT", "COST", "TGT", "HD"],
    "Aerospace & Defense": ["BA", "LMT", "NOC", "GD", "RTX"],
    "Telecom & Media": ["VZ", "T", "DIS", "NFLX", "CMCSA"],
    "Semiconductors": ["AMD", "TSM", "INTC", "AVGO", "QCOM"]
}

# Function to fetch sector-wise stock data
def get_sector_stock_data():
    sector_data = {}

    for sector, tickers in us_stock_universe.items():
        sector_prices = {"Last Price": [], "Open Price": [], "Highest": [], "Lowest": []}

        for ticker in tickers:
            stock = yf.Ticker(ticker)
            hist = stock.history(period="1d")  # Fetch latest daily data

            if not hist.empty:
                sector_prices["Last Price"].append(hist["Close"].iloc[-1])
                sector_prices["Open Price"].append(hist["Open"].iloc[-1])
                sector_prices["Highest"].append(hist["High"].iloc[-1])
                sector_prices["Lowest"].append(hist["Low"].iloc[-1])

        # Calculate average prices for the sector
        if sector_prices["Last Price"]:
            sector_data[sector] = {
                "Avg Last Price": sum(sector_prices["Last Price"]) / len(sector_prices["Last Price"]),
                "Avg Open Price": sum(sector_prices["Open Price"]) / len(sector_prices["Open Price"]),
                "Avg Highest": sum(sector_prices["Highest"]) / len(sector_prices["Highest"]),
                "Avg Lowest": sum(sector_prices["Lowest"]) / len(sector_prices["Lowest"]),
            }
        else:
            sector_data[sector] = "Data not available"

    return pd.DataFrame(sector_data).T

# Fetch sector data
sector_stock_data = get_sector_stock_data()

# Display and save the data
print(sector_stock_data)
sector_stock_data.to_csv("sector_data.csv", index=True)

print("✅ Sector-wise stock data saved to sector_stock_data.csv")
